# Test notebook

## Imports

In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import folium
import streamlit as st
import os
import matplotlib
from rasterstats import zonal_stats
import rasterio
import requests

from src.utils import *
from src.utils_population import *
from src.utils_plotting import *

## General parameters

In [42]:
verbose = True

## User input

In [43]:
# Parameters given by the user
# gdf_file = "test_data/mda_isochrones.geojson"
# gdf_file = 'test_data/mda_isochrones.zip'
gdf_file = 'test_data/shakeintensity_complex.zip'
data_type = "UNadj_constrained"
year = 2020
aggregated = False

## Process input data

In [44]:
# Create geopandas dataframes and prepare starting points
gdf, error = load_gdf(gdf_file)

if error:
    raise ValueError(error)
    
gdf = gdf.iloc[:5, ]

In [45]:
gdf

,id,grid_code,grid_value,mag,eventTime,updated,url,Shape__Are,Shape__Len,geometry
0,us6000jlqa,4,4.8,7.5,2023-02-06,2023-02-12,https://earthquake.usgs.gov/earthquakes/eventp...,7.495473e+11,7.218840e+07,"MULTIPOLYGON (((38.50207 32.83333, 38.48494 32..."
1,us6000jllz,4,4.8,7.8,2023-02-06,2023-02-12,https://earthquake.usgs.gov/earthquakes/eventp...,1.307015e+12,1.229430e+08,"MULTIPOLYGON (((35.95496 29.11667, 35.94378 29..."
2,us7000j56z,2,2.8,5.8,2023-01-18,2023-01-22,https://earthquake.usgs.gov/earthquakes/eventp...,9.668782e+10,3.006155e+07,"MULTIPOLYGON (((43.66047 36.58333, 43.65409 36..."
3,us7000j56z,3,3.8,5.8,2023-01-18,2023-01-22,https://earthquake.usgs.gov/earthquakes/eventp...,1.470852e+11,3.247177e+07,"MULTIPOLYGON (((44.24982 36.58351, 44.25000 36..."
4,us6000jk0t,2,2.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,5.373947e+10,2.705561e+07,"MULTIPOLYGON (((44.29176 36.60000, 44.29165 36..."


## Run computation

In [54]:
%%time

zonal_stat = add_population_data(
    gdf=gdf,
    data_type=data_type,
    tif_folder='test_data/pop_data',
    data_folder='data',
    year=year,
    aggregated=aggregated,
    clobber=True,
    verbose=True,
    text_on_streamlit=False,
    progress_bar=False,
)

Retrieving data from Google Earth Engine...
Band (1/36): M_0
Band (2/36): M_1
Band (3/36): M_5
Band (4/36): M_10
Band (5/36): M_15
Band (6/36): M_20
Band (7/36): M_25
Band (8/36): M_30
Band (9/36): M_35
Band (10/36): M_40
Band (11/36): M_45
Band (12/36): M_50
Band (13/36): M_55
Band (14/36): M_60
Band (15/36): M_65
Band (16/36): M_70
Band (17/36): M_75
Band (18/36): M_80
Band (19/36): F_0
Band (20/36): F_1
Band (21/36): F_5
Band (22/36): F_10
Band (23/36): F_15
Band (24/36): F_20
Band (25/36): F_25
Band (26/36): F_30
Band (27/36): F_35
Band (28/36): F_40
Band (29/36): F_45
Band (30/36): F_50
Band (31/36): F_55
Band (32/36): F_60
Band (33/36): F_65
Band (34/36): F_70
Band (35/36): F_75
Band (36/36): F_80
CPU times: user 1.91 s, sys: 75.8 ms, total: 1.98 s
Wall time: 36.9 s


In [ ]:
zonal_stat.getInfo()

## Plot

In [ ]:
aggregated=aggregated
joint=False
col_label = 'PARAMVALUE'
legend_title = 'Admin level'
plot_title = 'Admin level'

fig = plot_pop_data(
    gdf=gdf_with_pop, 
    col_label=col_label,
    legend_title=legend_title, 
    plot_title=plot_title,
    aggregated=aggregated,
    joint=joint,
    cmap_name='viridis'
    )

fig.show()